https://machinelearningmastery.com/understanding-stateful-lstm-recurrent-neural-networks-python-keras/

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
np.random.seed(7)

In [3]:
# define the raw dataset
phrase = "Machine learning lab: Redes Neuronales Recurrentes (LSTM)"
phrase = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
print('Longitud de la frase:',len(phrase))
# create mapping of characters to integers (0-25) and the reverse
def create_dicts(phrase):
    alphabet = sorted(list(set(phrase)))
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    return alphabet, char_to_int, int_to_char
alphabet, char_to_int, int_to_char = create_dicts(phrase)
print(len(alphabet))
print(char_to_int)
print(int_to_char)

Longitud de la frase: 26
26
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}


In [4]:
# prepare the dataset of input to output pairs encoded as integers
def prepare_dataset(phrase, seq_length = 1, verbose = True):
    alphabet, char_to_int, int_to_char = create_dicts(phrase)
    dataX = []
    dataY = []
    if verbose:
        print('Primeros 3 pares:')
    for i in range(0, len(phrase) - seq_length, 1):
        seq_in = phrase[i:i + seq_length]
        seq_out = phrase[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
        if verbose and i<3:
            print(seq_in, '->', seq_out)
        # reshape X to be [samples, time steps, features]
    X = np.array(dataX).reshape((len(dataX), seq_length, 1))
    # en X_mlp time steps sería features
    X_mlp = X.reshape(len(dataX), seq_length)
    
    X = X / float(len(alphabet))
    X_mlp = X_mlp / float(len(alphabet))
    # one hot encode the output variable
    y = np_utils.to_categorical(dataY)
    if verbose:
        print()
        print('X e y ya formateados para RNN:')
        print(X[0], '->', y[0])
        print(X[1], '->', y[1])
        print()
        print('Dimensiones de X para RNN [samples, time steps, features]')
        print(X.shape)
        print()
        print('Dimensiones de X_mpl para MLP [samples, features]')
        print(X_mlp.shape)
    
    return X, y, dataX, dataY, X_mlp

X, y, dataX, dataY, X_mlp = prepare_dataset(phrase, seq_length = 3, verbose=True)

Primeros 3 pares:
ABC -> D
BCD -> E
CDE -> F

X e y ya formateados para RNN:
[[ 0.        ]
 [ 0.03846154]
 [ 0.07692308]] -> [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
[[ 0.03846154]
 [ 0.07692308]
 [ 0.11538462]] -> [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]

Dimensiones de X para RNN [samples, time steps, features]
(23, 3, 1)

Dimensiones de X_mpl para MLP [samples, features]
(23, 3)


## MLP como baseline

In [8]:
def get_score(X, y, model): 
    # summarize performance of the model
    scores = model.evaluate(X, y, verbose=0)
    print("Model Accuracy: %.2f%%" % (scores[1]*100))

def get_some_model_predictions_MLP(model, dataX):
    # demonstrate some model predictions
    for pattern in dataX:
        x = np.array(pattern)
        x = x / float(len(alphabet))
        x = x.reshape(1,x.shape[0])
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        print(seq_in, "->", result)
        
def get_some_model_predictions(model, dataX):
    # demonstrate some model predictions
    for pattern in dataX:
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(len(alphabet))
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        print(seq_in, "->", result)

In [9]:
def get_MLP_Model():
    # create and fit the model
    model = Sequential()
    model.add(Dense(32, input_dim=X_mlp.shape[1]))
    model.add(Dense(y.shape[1], activation='softmax'))
    return model

In [16]:
model_MLP_1 = get_MLP_Model()
model_MLP_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                64        
_________________________________________________________________
dense_5 (Dense)              (None, 26)                858       
Total params: 922
Trainable params: 922
Non-trainable params: 0
_________________________________________________________________


In [10]:
seq_length = 1
X, y, dataX, dataY, X_mlp = prepare_dataset(phrase, seq_length = seq_length, verbose=True)

Primeros 3 pares:
A -> B
B -> C
C -> D

X e y ya formateados para RNN:
[[ 0.]] -> [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
[[ 0.03846154]] -> [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]

Dimensiones de X para RNN [samples, time steps, features]
(25, 1, 1)

Dimensiones de X_mpl para MLP [samples, features]
(25, 1)


In [11]:
model_MLP_1 = get_MLP_Model()
model_MLP_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_MLP_1.fit(X_mlp, y, epochs=500, batch_size=1, verbose=1)

Epoch 1/500
25/25 [==============================] - 1s - loss: 3.2935 - acc: 0.0400       
Epoch 2/500
25/25 [==============================] - 0s - loss: 3.2718 - acc: 0.0000e+00     
Epoch 3/500
25/25 [==============================] - 0s - loss: 3.2603 - acc: 0.0000e+00     
Epoch 4/500
25/25 [==============================] - 0s - loss: 3.2505 - acc: 0.0400       
Epoch 5/500
25/25 [==============================] - 0s - loss: 3.2424 - acc: 0.0400         
Epoch 6/500
25/25 [==============================] - 0s - loss: 3.2328 - acc: 0.0000e+00     
Epoch 7/500
25/25 [==============================] - 0s - loss: 3.2255 - acc: 0.0400        
Epoch 8/500
25/25 [==============================] - 0s - loss: 3.2171 - acc: 0.0000e+00     
Epoch 9/500
25/25 [==============================] - 0s - loss: 3.2093 - acc: 0.0400        
Epoch 10/500
25/25 [==============================] - 0s - loss: 3.2026 - acc: 0.0400         
Epoch 11/500
25/25 [==============================] - 0s - loss: 

25/25 [==============================] - 0s - loss: 2.5373 - acc: 0.1600      
Epoch 90/500
25/25 [==============================] - 0s - loss: 2.5337 - acc: 0.1600      
Epoch 91/500
25/25 [==============================] - 0s - loss: 2.5272 - acc: 0.1200       
Epoch 92/500
25/25 [==============================] - 0s - loss: 2.5231 - acc: 0.1200       
Epoch 93/500
25/25 [==============================] - ETA: 0s - loss: 2.5299 - acc: 0.1304  - 0s - loss: 2.5183 - acc: 0.1200     
Epoch 94/500
25/25 [==============================] - 0s - loss: 2.5140 - acc: 0.1600      
Epoch 95/500
25/25 [==============================] - 0s - loss: 2.5101 - acc: 0.1200       
Epoch 96/500
25/25 [==============================] - 0s - loss: 2.5053 - acc: 0.1600      
Epoch 97/500
25/25 [==============================] - 0s - loss: 2.4997 - acc: 0.1600     
Epoch 98/500
25/25 [==============================] - 0s - loss: 2.4966 - acc: 0.1200       
Epoch 99/500
25/25 [==============================]

25/25 [==============================] - 0s - loss: 2.2260 - acc: 0.3200      
Epoch 177/500
25/25 [==============================] - 0s - loss: 2.2218 - acc: 0.2800     
Epoch 178/500
25/25 [==============================] - 0s - loss: 2.2217 - acc: 0.2000       
Epoch 179/500
25/25 [==============================] - 0s - loss: 2.2200 - acc: 0.2800     
Epoch 180/500
25/25 [==============================] - 0s - loss: 2.2157 - acc: 0.2800     
Epoch 181/500
25/25 [==============================] - 0s - loss: 2.2139 - acc: 0.2400      
Epoch 182/500
25/25 [==============================] - 0s - loss: 2.2098 - acc: 0.2800     
Epoch 183/500
25/25 [==============================] - 0s - loss: 2.2085 - acc: 0.1200      
Epoch 184/500
25/25 [==============================] - 0s - loss: 2.2054 - acc: 0.3600     
Epoch 185/500
25/25 [==============================] - 0s - loss: 2.2036 - acc: 0.4000     
Epoch 186/500
25/25 [==============================] - 0s - loss: 2.2010 - acc: 0.2800   

25/25 [==============================] - 0s - loss: 2.0425 - acc: 0.4400      
Epoch 264/500
25/25 [==============================] - 0s - loss: 2.0398 - acc: 0.4400     
Epoch 265/500
25/25 [==============================] - 0s - loss: 2.0381 - acc: 0.4400       
Epoch 266/500
25/25 [==============================] - 0s - loss: 2.0370 - acc: 0.4000     
Epoch 267/500
25/25 [==============================] - 0s - loss: 2.0370 - acc: 0.3200     
Epoch 268/500
25/25 [==============================] - 0s - loss: 2.0340 - acc: 0.4800       
Epoch 269/500
25/25 [==============================] - 0s - loss: 2.0335 - acc: 0.4800       
Epoch 270/500
25/25 [==============================] - 0s - loss: 2.0283 - acc: 0.5200      
Epoch 271/500
25/25 [==============================] - 0s - loss: 2.0276 - acc: 0.4000       
Epoch 272/500
25/25 [==============================] - 0s - loss: 2.0268 - acc: 0.2800       
Epoch 273/500
25/25 [==============================] - 0s - loss: 2.0249 - acc: 0.

25/25 [==============================] - 0s - loss: 1.9180 - acc: 0.4400     
Epoch 351/500
25/25 [==============================] - 0s - loss: 1.9157 - acc: 0.4800     
Epoch 352/500
25/25 [==============================] - 0s - loss: 1.9134 - acc: 0.4800      
Epoch 353/500
25/25 [==============================] - 0s - loss: 1.9125 - acc: 0.5600     
Epoch 354/500
25/25 [==============================] - 0s - loss: 1.9132 - acc: 0.5600     
Epoch 355/500
25/25 [==============================] - 0s - loss: 1.9095 - acc: 0.5600      
Epoch 356/500
25/25 [==============================] - 0s - loss: 1.9067 - acc: 0.6000     
Epoch 357/500
25/25 [==============================] - 0s - loss: 1.9083 - acc: 0.6400     
Epoch 358/500
25/25 [==============================] - 0s - loss: 1.9062 - acc: 0.6000     
Epoch 359/500
25/25 [==============================] - 0s - loss: 1.9067 - acc: 0.6400     
Epoch 360/500
25/25 [==============================] - 0s - loss: 1.9060 - acc: 0.4800     


25/25 [==============================] - 0s - loss: 1.8234 - acc: 0.6800     
Epoch 439/500
25/25 [==============================] - 0s - loss: 1.8190 - acc: 0.6000     
Epoch 440/500
25/25 [==============================] - 0s - loss: 1.8194 - acc: 0.6800     
Epoch 441/500
25/25 [==============================] - 0s - loss: 1.8177 - acc: 0.6400      
Epoch 442/500
25/25 [==============================] - 0s - loss: 1.8159 - acc: 0.5600     
Epoch 443/500
25/25 [==============================] - 0s - loss: 1.8153 - acc: 0.5600       
Epoch 444/500
25/25 [==============================] - 0s - loss: 1.8161 - acc: 0.5600       
Epoch 445/500
25/25 [==============================] - 0s - loss: 1.8122 - acc: 0.6800     
Epoch 446/500
25/25 [==============================] - 0s - loss: 1.8121 - acc: 0.6800     
Epoch 447/500
25/25 [==============================] - 0s - loss: 1.8127 - acc: 0.5600      
Epoch 448/500
25/25 [==============================] - 0s - loss: 1.8140 - acc: 0.5600  

In [12]:
get_score(X_mlp, y, model_MLP_1)
get_some_model_predictions_MLP(model_MLP_1, dataX)

Model Accuracy: 84.00%
['A'] -> B
['B'] -> B
['C'] -> D
['D'] -> E
['E'] -> F
['F'] -> G
['G'] -> H
['H'] -> I
['I'] -> J
['J'] -> K
['K'] -> L
['L'] -> M
['M'] -> N
['N'] -> O
['O'] -> P
['P'] -> Q
['Q'] -> R
['R'] -> T
['S'] -> T
['T'] -> U
['U'] -> V
['V'] -> W
['W'] -> Y
['X'] -> Z
['Y'] -> Z


## LSTM

## Comenzamos con el caso mas simple posible:
- seq_length = 1
- batch_size = 1

In [13]:
def get_naive_LSTM():
    # create and fit the model
    model = Sequential()
    model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(y.shape[1], activation='softmax'))
    return model

In [18]:
model_naive = get_naive_LSTM()
model_naive.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_7 (Dense)              (None, 26)                858       
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
_________________________________________________________________


In [14]:
model_naive = get_naive_LSTM()
model_naive.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_naive.fit(X, y, epochs=500, batch_size=1, verbose=2)

Epoch 1/500
0s - loss: 3.2671 - acc: 0.0400
Epoch 2/500
0s - loss: 3.2595 - acc: 0.0400
Epoch 3/500
0s - loss: 3.2573 - acc: 0.0400
Epoch 4/500
0s - loss: 3.2547 - acc: 0.0400
Epoch 5/500
0s - loss: 3.2518 - acc: 0.0400
Epoch 6/500
0s - loss: 3.2494 - acc: 0.0400
Epoch 7/500
0s - loss: 3.2471 - acc: 0.0400
Epoch 8/500
0s - loss: 3.2444 - acc: 0.0400
Epoch 9/500
0s - loss: 3.2415 - acc: 0.0400
Epoch 10/500
0s - loss: 3.2387 - acc: 0.0400
Epoch 11/500
0s - loss: 3.2356 - acc: 0.0400
Epoch 12/500
0s - loss: 3.2320 - acc: 0.0400
Epoch 13/500
0s - loss: 3.2286 - acc: 0.0400
Epoch 14/500
0s - loss: 3.2250 - acc: 0.0400
Epoch 15/500
0s - loss: 3.2210 - acc: 0.0400
Epoch 16/500
0s - loss: 3.2164 - acc: 0.0400
Epoch 17/500
0s - loss: 3.2119 - acc: 0.0400
Epoch 18/500
0s - loss: 3.2076 - acc: 0.0400
Epoch 19/500
0s - loss: 3.2024 - acc: 0.0400
Epoch 20/500
0s - loss: 3.1965 - acc: 0.0400
Epoch 21/500
0s - loss: 3.1911 - acc: 0.0400
Epoch 22/500
0s - loss: 3.1851 - acc: 0.0400
Epoch 23/500
0s - l

Epoch 182/500
0s - loss: 2.1802 - acc: 0.2400
Epoch 183/500
0s - loss: 2.1757 - acc: 0.2000
Epoch 184/500
0s - loss: 2.1743 - acc: 0.2400
Epoch 185/500
0s - loss: 2.1713 - acc: 0.2800
Epoch 186/500
0s - loss: 2.1693 - acc: 0.3200
Epoch 187/500
0s - loss: 2.1679 - acc: 0.2400
Epoch 188/500
0s - loss: 2.1647 - acc: 0.2800
Epoch 189/500
0s - loss: 2.1609 - acc: 0.3200
Epoch 190/500
0s - loss: 2.1591 - acc: 0.2800
Epoch 191/500
0s - loss: 2.1553 - acc: 0.2800
Epoch 192/500
0s - loss: 2.1543 - acc: 0.3600
Epoch 193/500
0s - loss: 2.1529 - acc: 0.2800
Epoch 194/500
0s - loss: 2.1514 - acc: 0.2800
Epoch 195/500
0s - loss: 2.1475 - acc: 0.3200
Epoch 196/500
0s - loss: 2.1446 - acc: 0.3600
Epoch 197/500
0s - loss: 2.1430 - acc: 0.2800
Epoch 198/500
0s - loss: 2.1402 - acc: 0.2400
Epoch 199/500
0s - loss: 2.1380 - acc: 0.4400
Epoch 200/500
0s - loss: 2.1364 - acc: 0.3600
Epoch 201/500
0s - loss: 2.1330 - acc: 0.2800
Epoch 202/500
0s - loss: 2.1306 - acc: 0.2800
Epoch 203/500
0s - loss: 2.1286 - 

0s - loss: 1.8635 - acc: 0.6000
Epoch 361/500
0s - loss: 1.8601 - acc: 0.6800
Epoch 362/500
0s - loss: 1.8591 - acc: 0.7200
Epoch 363/500
0s - loss: 1.8579 - acc: 0.6400
Epoch 364/500
0s - loss: 1.8559 - acc: 0.5600
Epoch 365/500
0s - loss: 1.8564 - acc: 0.6400
Epoch 366/500
0s - loss: 1.8547 - acc: 0.6800
Epoch 367/500
0s - loss: 1.8523 - acc: 0.5600
Epoch 368/500
0s - loss: 1.8538 - acc: 0.6000
Epoch 369/500
0s - loss: 1.8516 - acc: 0.5600
Epoch 370/500
0s - loss: 1.8503 - acc: 0.5600
Epoch 371/500
0s - loss: 1.8451 - acc: 0.7200
Epoch 372/500
0s - loss: 1.8476 - acc: 0.6400
Epoch 373/500
0s - loss: 1.8466 - acc: 0.6400
Epoch 374/500
0s - loss: 1.8419 - acc: 0.6000
Epoch 375/500
0s - loss: 1.8433 - acc: 0.6000
Epoch 376/500
0s - loss: 1.8427 - acc: 0.6000
Epoch 377/500
0s - loss: 1.8393 - acc: 0.6400
Epoch 378/500
0s - loss: 1.8388 - acc: 0.5600
Epoch 379/500
0s - loss: 1.8372 - acc: 0.6800
Epoch 380/500
0s - loss: 1.8360 - acc: 0.7600
Epoch 381/500
0s - loss: 1.8353 - acc: 0.6800
Ep

In [15]:
get_score(X, y, model_naive)
get_some_model_predictions(model_naive, dataX)

Model Accuracy: 84.00%
['A'] -> B
['B'] -> C
['C'] -> D
['D'] -> E
['E'] -> F
['F'] -> G
['G'] -> H
['H'] -> I
['I'] -> J
['J'] -> K
['K'] -> L
['L'] -> M
['M'] -> N
['N'] -> O
['O'] -> P
['P'] -> Q
['Q'] -> R
['R'] -> S
['S'] -> T
['T'] -> U
['U'] -> W
['V'] -> X
['W'] -> Z
['X'] -> Z
['Y'] -> Z


## MLP 2

In [20]:
seq_length = 2
X, y, dataX, dataY, X_mlp = prepare_dataset(phrase, seq_length = seq_length, verbose=True)

Primeros 3 pares:
AB -> C
BC -> D
CD -> E

X e y ya formateados para RNN:
[[ 0.        ]
 [ 0.03846154]] -> [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
[[ 0.03846154]
 [ 0.07692308]] -> [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]

Dimensiones de X para RNN [samples, time steps, features]
(24, 2, 1)

Dimensiones de X_mpl para MLP [samples, features]
(24, 2)


In [ ]:

model_MLP = get_MLP_Model()
model_MLP.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_MLP.fit(X_mlp, y, epochs=500, batch_size=1, verbose=1)
get_score(X_mlp, y, model_MLP)
get_some_model_predictions_MLP(model_MLP, dataX)